In [1]:
import pickle
import pandas as pd
import pymysql
import json
import time
import requests
from tqdm import tqdm

### DB에 저장된 플레이어 정보 조회

In [2]:
conn = pymysql.connect(host='3.36.255.55', user='recipe', password='root1234', db='lol_project', charset='utf8')
cur = conn.cursor(pymysql.cursors.DictCursor)
sql = 'select puuid from challenger'
cur.execute(sql)
row = cur.fetchall()
conn.close()

### 피클로 저장된 게임 데이터 조회

In [11]:
with open('gameIdList.pickle','rb') as f:
    gameIdList = pickle.load(f)

### 인게임 데이터 API를 통해 받아오기

In [13]:
ingameList = []
api_key = 'RGAPI-21575c8f-4a5e-4e6b-97b8-4d39f8344206'
for gameId in tqdm(gameIdList):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + gameId + '?api_key=' + api_key
    datas = requests.get(url)
    datas = datas.json()
    time.sleep(1.7)
    ingameList.append(datas)
len(ingameList)

100%|████████████████████████████████████████████████████████████████████████████| 2968/2968 [1:41:05<00:00,  2.04s/it]


2968

### API로 받아온 데이터 DB에 저장 할 형태로 정리하기

In [74]:
def wd(x):
    if x == True:
        return 1
    else:
        return 0
inGameData, banList = [],[]
for datas in tqdm(ingameList):
    # 유저 개인당 인게임 데이터
    for pip in datas['info']['participants']:
        inGameData.append([datas['info']['gameId'],pip['kills'],pip['deaths'],pip['assists'],pip['champLevel'],pip['goldEarned'],pip['wardsKilled'],pip['wardsPlaced']
                           ,pip['visionWardsBoughtInGame'],pip['totalMinionsKilled'] + pip['neutralMinionsKilled']
                           ,pip['dragonKills'],pip['baronKills'],pip['turretKills'],pip['inhibitorKills']
                           ,pip['totalDamageDealtToChampions'],pip['totalDamageTaken']
                           ,pip['perks']['statPerks']['defense'],pip['perks']['statPerks']['flex'],pip['perks']['statPerks']['offense']
                           ,pip['perks']['styles'][0]['style']
                           ,pip['perks']['styles'][0]['selections'][0]['perk'],pip['perks']['styles'][0]['selections'][1]['perk']
                           ,pip['perks']['styles'][0]['selections'][2]['perk'],pip['perks']['styles'][0]['selections'][3]['perk']
                           ,pip['perks']['styles'][1]['style']
                           ,pip['perks']['styles'][1]['selections'][0]['perk'],pip['perks']['styles'][1]['selections'][1]['perk']
                           ,pip['item0'],pip['item1'],pip['item2'],pip['item3'],pip['item4'],pip['item5'],pip['item6']
                           ,pip['summoner1Id'],pip['summoner2Id']
                           ,pip['doubleKills'],pip['tripleKills'],pip['quadraKills'],pip['pentaKills']
                           ,pip['championId'],pip['summonerName'],pip['teamId'],pip['puuid'],pip['teamPosition'],wd(pip['win'])
                           ,datas['info']['gameDuration'],datas['info']['gameEndTimestamp']
                           ])
    for pip in datas['info']['teams']:
        for i in pip['bans']:
            banList.append(i['championId'])

plist = []
for gv,bv in zip(inGameData,banList):
    gv.append(bv)
    plist.append(bv)

plist = pd.DataFrame(inGameData, columns=['gameId','kills','deaths','assists','champLevel','gainGold','wardsKilled','wardsPlaced'
    ,'visionWards','cs','dragonKills','baronKills','turretKills','inhibitorKills','totalDamageDealt','totalDamageTaken'
    ,'defenseStat','flexStat','offenseStat','mainPerk0','mainPerk1','mainPerk2','mainPerk3','mainPerk4','subPerk0','subPerk1','subPerk2'
    ,'item0','item1','item2','item3','item4','item5','item6','summoner1Id','summoner2Id','doubleKills','tripleKills','quadraKills'
    ,'pentaKills','championId','summonerName','teamId','puuid','teamPosition','win','gameDuration','gameEndTimestamp','ban'])

100%|███████████████████████████████████████████████████████████████████████████| 2968/2968 [00:00<00:00, 10448.37it/s]


### 데이터 확인

In [61]:
plist[plist['gameId']==6180162894][['ban']]

,ban
0,-1
1,-1
2,266
3,238
4,555
5,555
6,91
7,104
8,-1
9,-1


In [102]:
plist.columns

Index(['gameId', 'kills', 'deaths', 'assists', 'champLevel', 'gainGold',
       'wardsKilled', 'wardsPlaced', 'visionWards', 'cs', 'dragonKills',
       'baronKills', 'turretKills', 'inhibitorKills', 'totalDamageDealt',
       'totalDamageTaken', 'defenseStat', 'flexStat', 'offenseStat',
       'mainPerk0', 'mainPerk1', 'mainPerk2', 'mainPerk3', 'mainPerk4',
       'subPerk0', 'subPerk1', 'subPerk2', 'item0', 'item1', 'item2', 'item3',
       'item4', 'item5', 'item6', 'summoner1Id', 'summoner2Id', 'doubleKills',
       'tripleKills', 'quadraKills', 'pentaKills', 'championId',
       'summonerName', 'teamId', 'puuid', 'teamPosition', 'win',
       'gameDuration', 'gameEndTimestamp', 'ban'],
      dtype='object')

### 데이터테이블 pickle에 저장하기

In [98]:
with open('inGameData.pickle','wb') as f:
    pickle.dump(plist,f)

### 정리된 데이터 DB에 저장하기

In [85]:
gameId = plist['gameId'].values
kills = plist['kills'].values
deaths = plist['deaths'].values
assists = plist['assists'].values
champLevel = plist['champLevel'].values
gainGold = plist['gainGold'].values
wardsKilled = plist['wardsKilled'].values
wardsPlaced = plist['wardsPlaced'].values
visionWards = plist['visionWards'].values
cs = plist['cs'].values
dragonKills = plist['dragonKills'].values
baronKills = plist['baronKills'].values
turretKills = plist['turretKills'].values
inhibitorKills = plist['inhibitorKills'].values
totalDamageDealt = plist['totalDamageDealt'].values
totalDamageTaken = plist['totalDamageTaken'].values
defenseStat = plist['defenseStat'].values
flexStat = plist['flexStat'].values
offenseStat = plist['offenseStat'].values
mainPerk0 = plist['mainPerk0'].values
mainPerk1 = plist['mainPerk1'].values
mainPerk2 = plist['mainPerk2'].values
mainPerk3 = plist['mainPerk3'].values
mainPerk4 = plist['mainPerk4'].values
subPerk0 = plist['subPerk0'].values
subPerk1 = plist['subPerk1'].values
subPerk2 = plist['subPerk2'].values
item0 = plist['item0'].values
item1 = plist['item1'].values
item2 = plist['item2'].values
item3 = plist['item3'].values
item4 = plist['item4'].values
item5 = plist['item5'].values
item6 = plist['item6'].values
summoner1Id = plist['summoner1Id'].values
summoner2Id = plist['summoner2Id'].values
doubleKills = plist['doubleKills'].values
tripleKills = plist['tripleKills'].values
quadraKills = plist['quadraKills'].values
pentaKills = plist['pentaKills'].values
championId = plist['championId'].values
summonerName = plist['summonerName'].values
teamId = plist['teamId'].values
puuid = plist['puuid'].values
teamPosition = plist['teamPosition'].values
win = plist['win'].values
gameDuration = plist['gameDuration'].values
gameEndTimestamp = plist['gameEndTimestamp'].values
ban = plist['ban'].values

In [97]:
conn = pymysql.connect(host='3.36.255.55', user='root', password='root1234', db='lol_project', charset='utf8')
cur = conn.cursor(pymysql.cursors.DictCursor)
for a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj,kk,ll,mm,nn,oo,pp,qq,rr,ss,tt,uu,vv,ww in zip(gameId,kills,deaths,assists,champLevel,gainGold,wardsKilled,wardsPlaced,visionWards,cs,dragonKills,baronKills,turretKills,inhibitorKills,totalDamageDealt,totalDamageTaken,defenseStat,flexStat,offenseStat,mainPerk0,mainPerk1,mainPerk2,mainPerk3,mainPerk4,subPerk0,subPerk1,subPerk2,item0,item1,item2,item3,item4,item5,item6,summoner1Id,summoner2Id,doubleKills,tripleKills,quadraKills,pentaKills,championId,summonerName,teamId,puuid,teamPosition,win,gameDuration,gameEndTimestamp,ban):
    sql = "insert into inGameData (gameId,kills,deaths,assists,champLevel,gainGold,wardsKilled,wardsPlaced,visionWards,cs,dragonKills,baronKills,turretKills,inhibitorKills,totalDamageDealt,totalDamageTaken,defenseStat,flexStat,offenseStat,mainPerk0,mainPerk1,mainPerk2,mainPerk3,mainPerk4,subPerk0,subPerk1,subPerk2,item0,item1,item2,item3,item4,item5,item6,summoner1Id,summoner2Id,doubleKills,tripleKills,quadraKills,pentaKills,championId,summonerName,teamId,puuid,teamPosition,win,gameDuration,gameEndTimestamp,ban) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    val = (a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj,kk,ll,mm,nn,oo,pp,qq,rr,ss,tt,uu,vv,ww)
    cur.execute(sql,val)
conn.commit()
conn.close()